In [6]:
import numpy as np
import pandas as pd
import sklearn as sk

import sklearn.preprocessing

import pymc3 as pm
import theano as T
import theano.tensor as tt

import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [3]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [4]:
query = \
"""
SELECT * FROM echo_filtered; 
"""
ef = pd.read_sql_query(query, con)
ef['mortality_30d'] = ef['ea_days_after_discharge_death'] <= 30
ef.head()

,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml,most_recent_hadm,first_echo,mortality_30d
0,67805,280987,182104,36,25443 days 10:29:14,M,NaN,NaN,NaN,WHITE,...,1116.0,508.0,1201.0,-693.0,NaN,NaN,NaN,1,1,False
1,67118,216609,116009,62,25097 days 00:20:44,M,68.0,150.0,22.804931,PATIENT DECLINED TO ANSWER,...,2083.0,505.0,1525.0,-1020.0,335.5,730.0,-394.5,1,1,False
2,80320,281607,112086,75,28018 days 10:35:50,F,NaN,NaN,NaN,WHITE,...,1630.0,12.0,1475.0,-1463.0,NaN,NaN,NaN,1,1,False
3,80083,206222,190243,86,16521 days 11:51:54,F,NaN,NaN,NaN,WHITE,...,1813.0,1006.0,1430.0,-424.0,NaN,NaN,NaN,1,1,False
4,82745,232514,114585,115,27643 days 13:22:28,F,63.0,207.0,36.664399,WHITE,...,2255.0,1105.5,1466.0,-360.5,NaN,NaN,NaN,1,1,False


In [10]:
data = ef.loc[ef['ea_lv_systolic'] >= -1,
        ['ea_lv_systolic', 'fl_day1_balance_ml', 
         'ex_congestive_heart_failure', 'mech_vent', 'apsiii', 
         'mortality_30d']].dropna()
data['ea_lv_systolic'] = (data['ea_lv_systolic'] + 2).astype(int)
data['mortality_30d'] = data['mortality_30d'].apply(lambda v: 1 if v else 0)
data['fl_day1_balance_ml'] = sk.preprocessing.scale(data['fl_day1_balance_ml'])
data['apsiii'] = sk.preprocessing.scale(data['apsiii'])
print(data.shape)
data.head()

(1716, 6)


/Users/yhtgrace/anaconda3/envs/patches/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


,ea_lv_systolic,fl_day1_balance_ml,ex_congestive_heart_failure,mech_vent,apsiii,mortality_30d
0,3,-0.213822,0,0,-1.139384,0
1,2,0.171148,0,0,0.000812,0
2,2,-0.009194,0,1,-0.759319,0
3,2,0.063659,0,0,-0.801548,0
4,1,0.239623,1,1,0.803173,0


In [18]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [21]:
def t_invlogit(x): 
    return T.tensor.exp(x) / (1 + T.tensor.exp(x))

with pm.Model() as model: 
    beta = pm.Normal('beta', mu = 0, sd = 10, shape = 5)
    p = t_invlogit(T.dot(data.iloc[:,:-1].values, beta))
    o = pm.Bernoulli('o', p, observed = y)
    
    trace = pm.sample(5000, njobs = 2)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average ELBO = nan: 100%|██████████| 200000/200000 [00:29<00:00, 6875.99it/s]
Finished [100%]: Average ELBO = nan


PositiveDefiniteError: Scaling is not positive definite. Simple check failed. Diagonal contains negatives. Check indexes [0 1 2 3 4]